In [7]:
def encrypt_caesar(message, key):
    result = ""
    for char in message:
        if char.isalpha():
            if char.isupper():
                result += chr((ord(char) + key - ord('A')) % 26 + ord('A'))
            else:
                result += chr((ord(char) + key - ord('a')) % 26 + ord('a'))
        else:
            result += char
    return result
def decrypt_caesar(encrypted_message, key):
    return encrypt_caesar(encrypted_message, -key)
if __name__ == "__main__":
    print("Caesar Encryption and Decryption")
    message_to_encrypt =input("Enter the Text to be Encrypted")
    caesar_key =int(input("Enter the Key Size"))
    encrypted_message = encrypt_caesar(message_to_encrypt, caesar_key)
    decrypted_message = decrypt_caesar(encrypted_message, caesar_key)
    print("Original Message:", message_to_encrypt)
    print("Encrypted Message:", encrypted_message)
    print("Decrypted Message:", decrypted_message)


Caesar Encryption and Decryption
Original Message: kumar
Encrypted Message: nxpdu
Decrypted Message: kumar


In [3]:
def encrypt_rail_fence(plaintext, rails):
    matrix = [['' for _ in range(len(plaintext))] for _ in range(rails)]
    row, col = 0, 0
    for char in plaintext:
        matrix[row][col] = char
        col += 1
        # Change direction when reaching the top or bottom rail
        if row == 0 or row == rails - 1:
            row += 1
        else:
            row -= 1
    # Read the matrix row by row to get the ciphertext
    ciphertext = ''.join(matrix[i][j] for j in range(len(plaintext)) for i in range(rails))
    return ciphertext
def decrypt_rail_fence(ciphertext, rails):
    matrix = [['' for _ in range(len(ciphertext))] for _ in range(rails)]
    row, col = 0, 0
    for char in ciphertext:
        matrix[row][col] = '*'
        col += 1
        # Change direction when reaching the top or bottom rail
        if row == 0 or row == rails - 1:
            row += 1
        else:
            row -= 1
    # Fill the matrix with the ciphertext
    index = 0
    for i in range(rails):
        for j in range(len(ciphertext)):
            if matrix[i][j] == '*' and index < len(ciphertext):
                matrix[i][j] = ciphertext[index]
                index += 1
    # Read the matrix column by column to get the plaintext
    plaintext = ''.join(matrix[i][j] for j in range(len(ciphertext)) for i in range(rails))
    return plaintext
# Example usage
plaintext = "Hello, "
rails = 3
# Encryption
ciphertext = encrypt_rail_fence(plaintext, rails)
print("Encrypted:", ciphertext)
# Decryption
decrypted_text = decrypt_rail_fence(ciphertext, rails)
print("Decrypted:", decrypted_text)



Encrypted: Hello, 
Decrypted: Hoe,l l


In [5]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

def generate_rsa_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_rsa(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def decrypt_rsa(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    ).decode()
    return plaintext

# Example usage
private_key, public_key = generate_rsa_key_pair()
message = "Hello, RSA!"

encrypted_message = encrypt_rsa(public_key, message)
print("Encrypted Message:", encrypted_message.hex())

decrypted_message = decrypt_rsa(private_key, encrypted_message)
print("Decrypted Message:", decrypted_message)


Encrypted Message: 2126834b2358e269a975fcc7d1627a85a1baf205d8a1ef4f86c0b14ba7832e27a574c425490a895a76b87c79a548aafd459b75f16b23cb256d5775f826ee4e31b2b0eab428134d460732689f7aef04570109a9169f1611cfbd389c342c3e54818b4a1d6d750a46985359f00906c8ca4f527d198bb2a4882348a81de1887646dfd4bab949250844d25af80f1ff37780a2d767ae6d8e427b65d433ffc6c2e938453d1760f39f4e43d9dd9b638b25303bcb552c9e548fc9c2d9b74d7717cf03be7f101a7e18543809f6457b63960108dff97512ded83f6a8095c34f66679732b18828b0eeb90f0feb88b660bbcfbebbf7411b8b0880d0da6b605975a2b35c0882bf
Decrypted Message: Hello, RSA!


In [9]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.backends import default_backend

def generate_dh_key_pair():
    parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def serialize_key(key):
    return key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

def deserialize_key(serialized_key):
    return serialization.load_pem_public_key(serialized_key, backend=default_backend())

def derive_shared_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    return shared_key

# Example usage
alice_private_key, alice_public_key = generate_dh_key_pair()
bob_private_key, bob_public_key = generate_dh_key_pair()

# Serialize public keys
serialized_alice_public_key = serialize_key(alice_public_key)
serialized_bob_public_key = serialize_key(bob_public_key)

# Deserialize public keys
alice_public_key = deserialize_key(serialized_bob_public_key)
bob_public_key = deserialize_key(serialized_alice_public_key)

# Derive shared keys
alice_shared_key = derive_shared_key(alice_private_key, bob_public_key)
bob_shared_key = derive_shared_key(bob_private_key, alice_public_key)

print("Alice's Shared Key:", alice_shared_key.hex())
print("Bob's Shared Key:", bob_shared_key.hex())


Alice's Shared Key: 16dfc0d42a1d12a9da5b66a32f0a4885ff4adf371a4a1ad811bd8cbcac51e46cc8af76f69a6640c8e2f342ed119a33e66edcff820a1d2b94ee90b907ee547409560b202c6d25e099913bf51210707885071d9a869465d7d8625460edd394779156312e2605ac21a7f2ef186210efbfb7d8041f12bf07891e72063302a3c3d97d76e87ff69fb52352c7002bc0448e4d5d862f02385c08af2292374e5f457711210e84278ea3a505b2c042af06bff8ef667e0806e2028541b2e1350c75728ad65832320e152ec69a8acde895c48e4d38608a79c0905e5fd63ed524950e74baabe35d2ff3be433a4cd109164d577357bd3377d0a60ce3ebce1646a2d1da5a0c1826
Bob's Shared Key: 0b7663a694ab7bfd96ec43ddc054a251e5619592dd64cb7f9e6cce2f7098370589232eeb2932704445d016bca6722d1597c66bceaef53f4e160e043d4c0a060e5aacbb45aa698251ce3753900f4056eb4c38dd0bd53ae4b7151482756d4983e18836d40c4d5d1f434c5dadb122c2b8591c5413ff33608a2a0ef8edcfa1909b2199bbbaa7285bd3a30eac799eca4632efac21c55090bb09aaa6d65315e35431d217ef27c884e4217e95adbc9bcf0ec899ad71105c2d226bdf713b7053c6565d3053939b0200292eea47e1f6a5baf826baaa785a40cd0183582f0bf1c0eac7044b2

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import ec

def generate_key_pair():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    signature = private_key.sign(
        message.encode(),
        ec.ECDSA(hashes.SHA256())
    )
    return signature

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message.encode(),
            ec.ECDSA(hashes.SHA256())
        )
        return True
    except Exception:
        return False

# Example usage
private_key, public_key = generate_key_pair()
message = "This is a signed message."

# Signing
signature = sign_message(private_key, message)
print("Signature:", signature.hex())

# Verifying
is_valid = verify_signature(public_key, message, signature)
print("Signature is valid:", is_valid)


ModuleNotFoundError: No module named 'cryptography'